# TAP x Pokemon

TAPxPokemon is a project that links togheter many technologies and applies them to the world of competitive Pokemon battling, processing battle data logs to extract useful information and tries to predict the outcome of a match using machine learning.

<img src="./img/pikachu_meme.png" alt="Best Pikachu Meme" title="Wow" width="800" />

# Project Introduction
### Technicalities First

<img src="./img/data_pipeline.png" title="Data Pipeline" width="1000" />

During this journey we are going to explore every data pipeline stage and its component, from *Ingestion* to *Visualization*.
But first...

# Data Source
## Where everything starts

# Pokemon Showdown
## [Showdown Battle Simulator](https://play.pokemonshowdown.com/)



As mentioned we are going to collect data regarding competitive pokemon battles, so we choose the best and most played competive simulator: Pokemon Showdown (aka SD).
The website doesn't have any API to use and the only way to obtain data is to deal with the messages passing through the websocket, briefly...a nightmare!

Thankfully a hero made an excellent job, not just with host to server communication but creating a battle-bot using 3 different AIs (based on their behavior), his name is pmariglia and his work is available at [pmariglia/showdown](https://github.com/pmariglia/showdown).
All I had to do was fixing the output, retrieving only the useful fields for the project and creating a json for every battle. Eventually everything

 As an example:
 ***
 `{
  "battle": [
    {
      "bid": 3,
      "player": "Norne42",
      "type": "gen8ou",
      "turns": 44
    }
  ],
  "pokemon": [
    {
      "name": "barraskewda",
      "ability": "swiftswim",
      "types": [
        "water"
      ],
      "item": "choiceband",
      "win": 1
    },
}` 
***
Shortened for obvious reasons.

In detail we are extracting data such as:
 - Battle Info (Player name, Turns, Type of Battle)
 - Pokemon Info (Name, Ability, Types, Moves, etc...)
 
<img src="./img/more_data.jpg" title="More Data" width="500" />

# Data Ingestion
## Where everything goes

Logstash is our choice for this project and there many reasons why:
 - Ease of use, in both coding and "Dockerizing"
 - Filters and parse data on the fly
 - Flume didn't work...
 
 <img src="./img/flume_logstash.jpg" title="Flume vs Logstash" width="300" />
 